![alt text](Python06-iterators/logo.png) 

# Итераторы, генераторы, декораторы

Тема непростая, будем слегка жертвовать формализмом и точностью в пользу понятности, а существование некотрых вещей вообще проигнорируем. Те из вас, кто захотят больше деталей и чётких формулировок найдут их в избытке в документации и интернете.

# Итераторы

Давайте вспомним цикл for, который мы прошли во второй лекции (основы питона 2), и еще раз на него посмотрим

In [1]:
for var in range(5):
    print (var)

0
1
2
3
4


In [2]:
for letter in 'Hello world':
    print (letter)

H
e
l
l
o
 
w
o
r
l
d


In [3]:
for key in {'Name':'John', 
            'Age':15, 
            'Gender':'Male', 
           }:
    print (key)

Name
Age
Gender


In [4]:
for line in open('Python06-iterators/motivation.txt', 'r'):
    print (line)

Today we will have an outstanding topic!

Iterators are amazing!!

You'll love it!!



In [5]:
for element in [0,1,2,3,4,5]:
    print (element)

0
1
2
3
4
5


То есть мы что, можем идти циклом по любому объекту? А это вообще законно?

Давайте разбираться, как мы помним из третьей лекции (ООП) - все объекты являются классами. Рассмотрим их методы

In [6]:
dir(range(5))

['__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'count',
 'index',
 'start',
 'step',
 'stop']

In [7]:
dir({'Name':'John', 
            'Age':15, 
            'Gender':'Male', 
           })

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

Метод  *\_\_iter*\_\_  возвращает итератор для заданного объекта. ![alt text](Python06-iterators/iterator.jpg) 

Итератор - это такой объект, который имплементирует протокол итерирования. Если сказать попроще, то итератор помнит текущее состояние и знает как выбирать следующий элемент объекта, над которым он построен.

In [8]:
a = [0,1,2,3,4,5,6,7,8,9]
my_iter = iter(a)
print(my_iter)

Метод *\_\_next*\_\_ возвращает объект соответствующий текущему состоянию итератора и переходит к следующему, согласно заданному алгоритму (в тривиальном случае - k+1й объект). Делать это можно до тех пор, пока не получим ислючение **StopIteration**

In [9]:
print(my_iter.__next__()) # Выглядит жутковато

0


In [10]:
print(next(my_iter)) # Вроде как-то попривычнее

1


In [11]:
# Будем делать так, чтобы уменьшить синтаксическую нагрузку
next(my_iter) 

2

Важное свойство итератора - он не хранит объект в памяти. 

In [12]:
my_object = [0,1,2,3,4,5,6,7,8,9]
my_iter = iter(my_object)
print(next(my_iter))
print(next(my_iter))

0
1


In [13]:
# Подменим объект не трогая итератор
my_object[2] = 'let put a string here'
my_object[3] = 'iterator does not care'

In [14]:
print(next(my_iter))

let put a string here


Метод  *\_\_iter*\_\_ может возвращать self (т.е. себя), то есть итератор от итератора - тоже итератор, причём тот же самый. ![alt text](Python06-iterators/more_iterators.jpg) 

In [15]:
a = [0,1,2,3,4,5]
my_iter = iter(iter(a))

In [16]:
next(my_iter)

0

Если быть достаточно наблюдательным, то можно заметить, что метод *\_\_next*\_\_ отсутствует у наших исходных объектов. Однако он появляется когда мы делаем из него итератор

In [17]:
dir(a)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [18]:
dir(iter(a))

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__length_hint__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [19]:
# Как проверить объект на итерируемость
my_object = [1,2,3]
#my_object = 2
#my_object = 'hello'
try:
    iter(my_object)
except TypeError:
    print('not iterable')
else:
    print('iterable')

iterable


Заметим, что итерироваться по объекту можно не только при помощи iter и next, но ещё и иначе, посредством метода *\_\_getitem*\_\_. Мы не будем его подробно рассматривать, отметим только что в отличии от iter который хранит состояние внутри итератора, getitem его не хранит, принимая его снаружи, со всеми вытекающими достоинствами и недостатками. Раньше это был основной протокол итерирования (в python<2.4), теперь, видимо, можно считать его устаревшим, но всё еще используемым.

In [20]:
a = [12,13,14,15,16,17]
b = a.__getitem__(3)
print(b)

15


Давайте рассмотрим **range**, это даст нам новый взгляд на итераторы. Допустим, нам надо проитерироваться по какому-то очень большому циклу. Втупую, для этого пришлось бы записать всю последовательность элементов в память, создать итератор, и идти от предыдущего к следующему много раз. Но держать в памяти этот огромный объект не слишком-то хотелось бы. Тут нас и выручают итераторы, которые не держат в памяти объект, а знают только своё текущее состояние и закон по которому выбираем следующий элемент.

In [21]:
range(5) # range не существует в каком-то полном запомненном виде

range(0, 5)

In [22]:
a = range(5)
# Но range на самом деле не итератор, это итерируемый объект
next(a)

TypeError: 'range' object is not an iterator

In [23]:
b = iter(range(5)) # А вот это - итератор по range

In [24]:
next(b)

0

In [25]:
# Давайте проверим, а вдруг теперь когда мы сделали из range итератор, там всё же записали весь объект
iter(range(5))

Ну, раз уж мы знаем ООП, давайте напишем свой класс с итератором. Это поможет нам лучше понять на что надо обращать внимание и как вообще их писать 

In [26]:
class NaiveRange():
    def __init__(self, start, end):
        self.end = end
        self.ind = start
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.ind >= self.end:
            raise StopIteration
        self.ind += 1
        return (self.ind - 1)

In [27]:
a = NaiveRange(3,9)

In [28]:
list(a)

[3, 4, 5, 6, 7, 8]

In [29]:
list(a)

[]

Дело в том, что архитектура нашего класса не позволяет индексу вернуться к исходному состоянию без ре-инициализации, он запомнен и с этим ничего не поделать. Такая итерация называется **исчерпаемой**. Пример: итерация по файлу. Мы можем его открыть и по мере прочтения достигнем конца. Конечно есть всякие трюки, но базово мы можем прочитать файл только раз, далее должны открыть его заново (и ре-инициализировать итератор).

Давайте теперь построим неисчераемый итератор.

In [30]:
class BaseIter(object):
    def __init__(self, start, end):
        self.end = end
        self.ind = start
        
    def __next__(self):
        if self.ind == self.end:
            raise StopIteration
        self.ind += 1
        return (self.ind -1)
    
    
class AdvancedRange(object):
    def __init__(self, start, end):
        self.end = end
        self.start = start
    def __iter__(self):
        return BaseIter(self.start, self.end)

In [31]:
a = AdvancedRange(3,9)

In [32]:
list(a)

[3, 4, 5, 6, 7, 8]

In [33]:
list(a)

[3, 4, 5, 6, 7, 8]

Теперь, при вызове итератора, мы создаем объект класса BaseIter и можем так делать сколько угодно раз, и каждый из них будет держать в своей памяти своё состояние, не мешая остальным итерироваться. Это, соответственно, **неисчерпаемая** итерация. Пример: список, словарь.

Теперь, с учётом полученных сведений, давайте вернёмся к тому с чего начинали, к циклу *for*

```python
some_iter = iter(some_object)
while True:    
    try:        
        some_value = next(some_iter)        
    except StopIteration:        
        break        
    do_smth(some_value)
```

# Генераторы

Идея следующая. Сделаем такую функцию, которая будет по какому-то входному параметру создавать итераторы. Ключевой является команда **yield**, которая работает примерно как *return* только при следующем обращении проход по телу будет осуществляться не с начала, а с места где мы закончили. Разберём пару примеров и станет ясно зачем это всё надо. 
![alt text](Python06-iterators/iter_vs_gen.svg)

In [34]:
def my_first_genertor(var):
    yield 'first launch'
    yield 'second launch'
    some_result = var*3+2
    yield some_result
    yield 'last launch'
    print ('Done, no more yields, only an exception - StopIteration')

In [35]:
gen = my_first_genertor(2)
#list(gen)

In [36]:
next(gen)

'first launch'

Далее, сколько бы мы не вызывали, если не ре-инициализировать генератор, то он больше ничего не выдаст. Генератор **исчерпался**. 

**Return** же в свою очередь в генераторе не работает как обычно, а работает исключительно как *StopIteration*. Если return не написать, то он подразумевается в конце.

In [37]:
def my_second_genertor(var):
    yield 'first launch'
    return 'second launch' # Заменим yield на return
    some_result = var*3+2
    yield some_result
    yield 'last launch'
    print ('Done, no more yields, only an exception - StopIteration')

In [38]:
gen = my_second_genertor(2)
#list(gen)

In [39]:
next(gen)

'first launch'

Заметим, что запустив повторно мы больше не переходим к следующему yield. Генератор исчерпался по исключению, не зависимо от того есть ли там ещё yield. 

**Важное замечание** Кроме описанного обычного завершения, стандартные генераторы можно завершить снаружи при помощи метода close. Этот метод вызывается, в том числе, принудительной остановкой и работает через исключение *GeneratorExit*. При самостоятельной реализации метода close, надо учитывать, что **GeneratorExit не отлавливается как базовое исключение**, и если не прописать его в явном виде при имплементации своего генератора, то есть шанс утратить возможность принудительной остановки, и никакие *Ctrl+C* не отработают. Уже после принудительной остановки, дальнейшие вызовы также получат *StopIteration*.

Выглядеть должно как-то так. Выглядит усложненным, но это близко к настоящей реализации.
```python
def safe_gen():
    closed = False
    try:
        yield 'check '
    except GeneratorExit:
        print('exit!')
        closed = True
        raise StopIteration
    finally:
        if not closed:
            print('worked!')

```

Зачем же нам всё это может нам понадобиться? Давайте вспомним одну классную *штуку* которую мы видели на второй лекции

In [41]:
nums = ['zero', 'one', 'two', 'three', 'four']
for index, number in enumerate(nums):
    print(index, number)

0 zero
1 one
2 two
3 three
4 four


**Enumerate** при итерации по объекту присваивает порядковые номера объектам, очень удобно и рекомендуется к использованию. *Enumerate* ни что иное как генератор, и может быть записано следующим образом

In [42]:
def my_enumerate(iterable_object):
    ind = 0
    for smth in iterable_object:
        yield ind, smth
        ind += 1

In [43]:
for index, number in my_enumerate(nums):
    print(index, number)

0 zero
1 one
2 two
3 three
4 four


Когда встречаешь что-то такое первый раз, то невольно возникает мысль "Ну написали цикл в функции, и зачем это всё? Я и сам могу цикл написать и безо всяких генераторов". Однако, стоит обратить внимание на то, что генератор не держит в памяти полностью записанный объект, вместо этого просто обращается к методу *next*, и это очень важно.

In [44]:
print(my_enumerate(nums))

<generator object my_enumerate at 0x7fbf70463c50>


Давайте рассмотрим ещё несколько генераторов

In [45]:
# map(func, smth_iterable) применяет к каждом элементу функцию func
double = map(lambda x: x + x, nums) 
print(list(double))

['zerozero', 'oneone', 'twotwo', 'threethree', 'fourfour']


In [46]:
# Потренируемся, сделаем свой map
def my_map(func, iterable_object):
    for smth in iterable_object:
        yield func(smth)

In [47]:
my_double = my_map(lambda x: x + x, nums) 
print(list(my_double))

['zerozero', 'oneone', 'twotwo', 'threethree', 'fourfour']


In [48]:
# filter(cond, smth_iterable) применяет к каждому элементу проверку по условию cond и отфильтровывает
len_filter = filter(lambda x: len(x) > 3, nums)
print(list(len_filter))

['zero', 'three', 'four']


In [49]:
# Cделаем свой filter
def my_filter(condition, iterable_object):
    for smth in iterable_object:
        if condition(smth):
            yield smth

In [50]:
# filter(cond, smth_iterable) применяет к каждому элементу проверку по условию cond и отфильтровывает
my_len_filter = my_filter(lambda x: len(x) > 3, nums)
print(list(my_len_filter))

['zero', 'three', 'four']


Есть множество полезных генераторов, которые пока ещё не впилили в основной питон (но судя по логам версий - этим занимаются): count, repeat, cycle, chain и другие. Их можно найти в пакете itertools. Давайте посмотрим.

In [51]:
from itertools import repeat
# Повторяет то что ему дали

In [ ]:
for var in repeat(5):
    print(var)

In [52]:
from itertools import chain
# "Раскрывает" итерируемые объекты и делает из них один. Но делает это только на 1 уровень в "глубину"

In [53]:
list(chain('A', 'B', [1, 2, 3, None], 'a string', range(4,7)))

['A', 'B', 1, 2, 3, None, 'a', ' ', 's', 't', 'r', 'i', 'n', 'g', 4, 5, 6]

In [54]:
list(chain('A', 'B', [1, 2, 3, 'a string', None], [range(4,7)]))

['A', 'B', 1, 2, 3, 'a string', None, range(4, 7)]

In [ ]:
from itertools import count
# Принимает начальное состояние и шаг и делает с ними бесконечный цикл

In [ ]:
for i in count(0,3):
    print(i)

Давайте еще рассмотрим такой пример. Допустим у нас есть объект, и мы хотим его перегруппировать в процессе итерации. 

In [55]:
my_object = [0, 1, 2, 'zero', 'one', 'two', 'A', 'B', 'C']
my_iter = iter(my_object)
print(list(zip(my_iter, my_iter, my_iter)))

[(0, 1, 2), ('zero', 'one', 'two'), ('A', 'B', 'C')]


### Выражения-генераторы

Стандарт Pep-289 даёт нам подробные инструкции как надо записывать и вообще как пользоваться выражениями-генераторами https://www.python.org/dev/peps/pep-0289/ 
Например,

In [56]:
def gen(exp):
    for x in exp:
        yield x**2
g = gen(iter(range(10)))

# Эквивалентно
g = (x**2 for x in range(10))

Это удобно краткой записью в одну строку, при этом это не просто выражение, а генератор, со всеми вытекающими преимуществами и свойствами, которые мы ранее изучили. 

**Расширим функциональность генератора**. Теперь мы хотим не только получать из него следующий элемент по команде, но и как-то более явно в этом участвовать, например посылать в него данные, чтобы получать другие данные. Для этого Используем команду **send**, а принимать данные будем при помощи всё того же *yield*, но немного иначе.

In [57]:
def gen():
    for i in range(10):
        x = yield 
        print(i + x)

In [58]:
G = gen()
next(G)

In [59]:
G.send(85)

85


In [60]:
# next теперь не работает
next(G)

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

![alt text](Python06-iterators/deeper.png) 

Давайте теперь рассмотрим такой пример, когда у нас есть два генератора, которые общаются между собой 

In [61]:
import random

# Генерим новые данные
def get_data():
    return random.sample(range(10), 2)

# Обрабатываем данные
def consume():
    total_sum = 0
    total_amount = 0
    
    while True:
        data = yield
        total_sum += sum(data)
        total_amount += len(data)
        print('Running average is {}'.format(total_sum/total_amount))
        
# Производим новые данные и передаём в обработку
def produce(consumer):
    while True:
        data = get_data()
        print('New data {}'.format(data))
        consumer.send(data)
        yield

In [62]:
consumer = consume()
next(consumer)
producer = produce(consumer)

for _ in range(1000):
    next(producer)

New data [9, 1]
Running average is 5.0
New data [3, 8]
Running average is 5.25
New data [2, 9]
Running average is 5.333333333333333
New data [6, 8]
Running average is 5.75
New data [0, 3]
Running average is 4.9
New data [8, 2]
Running average is 4.916666666666667
New data [4, 7]
Running average is 5.0
New data [8, 1]
Running average is 4.9375
New data [3, 8]
Running average is 5.0
New data [3, 1]
Running average is 4.7
New data [5, 4]
Running average is 4.681818181818182
New data [4, 7]
Running average is 4.75
New data [6, 9]
Running average is 4.961538461538462
New data [7, 0]
Running average is 4.857142857142857
New data [2, 7]
Running average is 4.833333333333333
New data [2, 5]
Running average is 4.75
New data [6, 7]
Running average is 4.852941176470588
New data [3, 6]
Running average is 4.833333333333333
New data [2, 3]
Running average is 4.7105263157894735
New data [1, 3]
Running average is 4.575
New data [3, 4]
Running average is 4.523809523809524
New data [9, 7]
Running average

Чем хороша приведённая конструкция (её идея)? Мы не просто разделили процесс генерации и обработки данных, это можно было сделать и без всяких генераторов. **Send** создаёт отдельный поток вычислений, которые производятся параллельно основному телу программы. Таким образом, генерить и обрабатывать данные многими разными способами можно в параллели, притормаживая только по ключевому слову *yield* чтобы дождаться данных или следующего обращения. 

Теперь, можем сказать что для генераторов next(a) эквивалентент a.send(None).

Всё это называется Coroutine и поподробнее можно почитать в PEP-342 https://www.python.org/dev/peps/pep-0342/

![alt text](Python06-iterators/coroutine.jpg)

Эта идея позволяет нам, например, создать планировщик задач, который по определенным условиям будет запускать тот или иной генератор, и экономить время выполняя их в параллели и переключаясь на них только когда это требуется. 

*Те кто переживает что плохо понял тему, и кто хочет разобраться и закрепить в памяти материал по генераторам, могут после лекции попробовать написать генератор, который в параллели качнет наши прошедшие лекции, обложить таймерами, и убедиться что это действительно работает быстрее. Это не обязательное задание, но это поможет усвоить материал. Ничего никуда посылать не надо*

Последнее, что хотелось бы рассмотреть это конструкция **yield from**. Обычно говорят, что эта конструкция устанавливает двунаправленную связь между вызывающим генератором и суб-генератором, передавая, в том числе, исключения (transparent two way channel).

Сначала простой пример:

In [63]:
def generator_simple():
    for i in range(10):
        yield i
    for j in range(10, 20):
        yield j

Он, будет выдавать нам числа от 0 до 19. И мы решили разбить его на две части чтобы их можно было еще использовать по отдельности.

In [64]:
def generator1():
    for i in range(10):
        yield i

def generator2():
    for j in range(10, 20):
        yield j
        
def generator():
    for i in generator1():
        yield i
    for j in generator2():
        yield j

In [65]:
# То же самое можно записать в виде
def generator_from():
    yield from generator1()
    yield from generator2()

Понятно, что это не очень впечатляет, так что давайте посмотрим чуть более сложный пример

In [66]:
def writer():
    """A coroutine that writes data *sent* to it """
    while True:
        w = (yield)
        print('>> ', w)

def writer_wrapper(coro):
    coro.send(None)  # Для инициализации
    while True:
        try:
            x = (yield)  # Принимаем
            coro.send(x)  # и передаём
        except StopIteration:
            pass

In [67]:
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)
for i in range(4):
    wrap.send(i)

>>  0
>>  1
>>  2
>>  3


In [68]:
#Всё то же самое выполняется при помощи yield from
def writer_wrapper_from(coro):
    yield from coro

In [69]:
w = writer()
wrap = writer_wrapper_from(w)
wrap.send(None)
for i in range(4):
    wrap.send(i)

>>  0
>>  1
>>  2
>>  3


Если подытожить, то yield from позволяет серьезно упростить код и не думать о всяких вещах, о которых мы обычно не хотим думать, и **не теряет** при этом никаких важных исключений или типа того.

# Декораторы

Допустим, мы столкнулись с задачей, что нам надо подправить кое-какие функции, но лезть в них и менять код прямо там нам не сильно охота, потому что, например, мы можем потом передумать. Или переживаем за общую совместимость в большом проекте. Сделаем обёртку нужной нам функции:

In [70]:
def deprecated(func):
    def wrap(*args, **kwargs):
        print ('{} will be deprecated soon'.format(func.__name__))
        return func(*args, **kwargs)
    return wrap

In [71]:
def my_func(x):
    return x+3

# "Обернём" нашу функцию посредством переприсвоения
my_func = deprecated(my_func)

In [72]:
my_func(2)

my_func will be deprecated soon


5

In [73]:
# Ровно то же самое можно записать без переприсвоения:
@deprecated # Это декоратор
def my_func(x):
    return x+3

In [74]:
my_func(2)

my_func will be deprecated soon


5

In [75]:
# Но, заметим, что именем ф-ии стало имя обёртки
my_func.__name__

'wrap'

Чтобы сделать красивый декоратор и совсем ничего в функции не менять, кроме того что мы поменять хотим, давайте воспользуемся библиотекой functools ![alt text](Python06-iterators/decoration.jpeg)

In [76]:
import functools

def deprecated(func):
    @functools.wraps(func)
    def wrap(*args, **kwargs):
        print ('{} will be deprecated soon'.format(func.__name__))
        return func(*args, **kwargs)
    return wrap

@deprecated
def my_func(x):
    return x+3

my_func.__name__

'my_func'

Functools - высокоуровневая библиотека, упрощающая работу с функциями и другими вызываемыми объектами, когда требуется расширить, дополнить или даже полностью изменить сам объект. Более детально можно познакомиться, например, здесь https://docs.python.org/3.3/library/functools.html

Декораторы работают с классами так же как и с функциями. Создадим декоратор выводящий время работы декорируемой функции, простой класс производящий какую-то операцию, наложим и посмотрим

In [77]:
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer


In [78]:
class TimeWaster:    
    def __init__(self, max_num):
        self.max_num = max_num

    #@timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])        

In [79]:
tw = TimeWaster(1000)
tw.waste_time(999)

Допустим, мы хотим чтобы наш декоратор что-то нам важное вывел, например исключение. Только не в общий вывод, чтобы не засорять, а куда-то в другое место, где мы при необходимости прочитаем. Для этого можно использовать атрибуты декоратора

In [80]:
def redirected_output(destination=None):        
    def wrapper(*args, **kwargs):
        with open(destination, 'w') as output:
            print('Some important information', 
                  file=output)    
    return wrapper

In [81]:
@redirected_output(destination='log.txt')
def foo():
    pass

Этот пример не очень хорош, зато он прост и демонстрирует, что в сам декоратор можно передавать входные данные

![alt text](Python06-iterators/end.jpg)